# OCR
This project implements OCR for the first 10 pages of a general PDF. This program will be compatiable with SQL.

### Things to Note
- to use a juypter notebook in vs code through uv, you must do these things:
    - uv init the project
    - create a venv
    - uv add --dev ipykernel
    - in vs code when selecting a kernel for the notebook, select pyhton then your venv
- Using `easyocr`
    - in order to read pdf documents, I've had to using the package pdf2image which requires poppler binary files which are attached in this project
    - this introduces system a system depency with the bin files
    - also I've been experiencing some crashing on my computer due to the model running on my cpu I think
    - I have a gpu though I'm not exactly sure how to get it to run on there
    - it would be most ideal if I could use google colabs gpus (I already have an account with them)

### Next Steps
- increase computation speed (switch to gpu)
- visualize which words are read on pdf (box detected letters)
- add compatability with SQL

In [ ]:
import sys
# check env
print(sys.executable)

c:\Users\dsbor\OneDrive\Desktop\Personal\Davis-PDF-Extraction\.venv\Scripts\python.exe


In [3]:
import easyocr

reader = easyocr.Reader(['en'], gpu=True)
result = reader.readtext('good_ex.jpg')

for (bbox, text, prob) in result:
    print(result)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[np.int32(176), np.int32(433)], [np.int32(853), np.int32(433)], [np.int32(853), np.int32(662)], [np.int32(176), np.int32(662)]], 'OPEN', np.float64(0.9998870310870229))]


c:\Users\dsbor\OneDrive\Desktop\Personal\Davis-PDF-Extraction\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


In [ ]:
from pdf2image import convert_from_path
import easyocr
import numpy as np

pages = convert_from_path('example.pdf', poppler_path=r'poppler_bin', first_page=1, last_page=2)

reader = easyocr.Reader(['en'], gpu=True)

for i, page_image in enumerate(pages, start=1):
    # convert PIL image to np array
    img_np = np.array(page_image)

    result = reader.readtext(img_np)

    print(f"\n--- Text on page {i} ---")
    for bbox, text, prob in result:
        print(f'Text: {text}, Probability: {prob:.4f}')

Using CPU. Note: This module is much faster with a GPU.
c:\Users\dsbor\OneDrive\Desktop\Personal\Davis-PDF-Extraction\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)



--- Text on page 1 ---
Text: Rayne, Probability: 0.6247
Text: Nutrition, Probability: 1.0000
Text: Product, Probability: 0.9999
Text: Catalog, Probability: 1.0000
Text: 2025 Edition, Probability: 0.8667
Text: RAYNE NUTRITION 2025, Probability: 0.8286

--- Text on page 2 ---
Text: Our Philosophy, Probability: 0.9994
Text: Our Whole Food Philosophy is based in the data that less-processed (aka "whole, Probability: 0.8270
Text: food") ingredients are the key to making foods that increase the quality and length, Probability: 0.7783
Text: of a, Probability: 0.9343
Text: life., Probability: 0.6962
Text: Dogs and, Probability: 0.9956
Text: cats have certain nutrients they need every, Probability: 0.8976
Text: to, Probability: 0.9998
Text: meet their needs,, Probability: 0.8692
Text: promote their health, and improve their longevity: Where those nutrients come from, Probability: 0.7235
Text: matters. When the nutrients in a diet are provided individually through, Probability: 0.7881
Text: sup

MemoryError: Unable to allocate 43.7 MiB for an array with shape (2208, 1728, 3) and data type float32